In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from getpass import getpass
from pathlib import Path
import geopandas as gpd
from shapely.geometry import Polygon

In [2]:
# connect to the API
api = SentinelAPI(input('user: '), getpass('password: '), 'https://apihub.copernicus.eu/apihub')

user: joaofonseca
password: ········


In [3]:

lat_list = [ 25.578848,  27.059566,  27.472921,  25.995016,  25.578848]
lon_list = [-15.385184, -15.063815, -17.591578, -17.880167, -15.385184]

polygon_geom = Polygon(zip(lon_list, lat_list))
crs = 'epsg:4326'
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])  

json_dir = '../assets/test_polygon.geojson'
polygon.to_file(filename=json_dir, driver='GeoJSON')
# polygon.to_file(filename='polygon.shp', driver="ESRI Shapefile")

In [4]:
footprint = geojson_to_wkt(read_geojson(json_dir))
products = api.query(footprint,
                     producttype='GRD',
                     orbitdirection='DESCENDING',
                     date = ('20221005', date(2022, 10, 7)),
                     limit=10)

In [5]:
# download single scene by known product id
product_id = list(products.keys())[1]
print('Product ID', product_id)
out_dir = Path('../assets/products/')
# Download all products found by query
# api.download_all(products)
# Download single product by id
api.download(product_id, directory_path=out_dir)

Product ID 418d5f53-f90a-4124-b2ce-0a685c1b54d7


MD5 checksumming:   0%|          | 0.00/753M [00:00<?, ?B/s]

{'id': '418d5f53-f90a-4124-b2ce-0a685c1b54d7',
 'title': 'S1A_IW_GRDH_1SDV_20221006T070325_20221006T070349_045318_056B02_53E7',
 'size': 752671268,
 'md5': '73e8a26a4595eb4e1d8f44d898440250',
 'date': datetime.datetime(2022, 10, 6, 7, 3, 25, 125000),
 'footprint': 'POLYGON((-15.385184 25.578848,-17.880167 25.995016,-17.591578 27.472921,-15.063815 27.059566,-15.385184 25.578848))',
 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('418d5f53-f90a-4124-b2ce-0a685c1b54d7')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2022, 10, 6, 7, 43, 8, 15000),
 'Ingestion Date': datetime.datetime(2022, 10, 6, 7, 42, 51, 829000),
 'quicklook_url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('418d5f53-f90a-4124-b2ce-0a685c1b54d7')/Products('Quicklook')/$value",
 'path': '../assets/products/S1A_IW_GRDH_1SDV_20221006T070325_20221006T070349_045318_056B02_53E7.zip',
 'downloaded_bytes': 752671268}

In [6]:
from zipfile import ZipFile

# Unzip the S1 product
with ZipFile(list(Path(out_dir).glob('*.zip'))[0], 'r') as s1_product:
    s1_product.extractall(out_dir)